In [1]:
from wayne_utils import load_data, save_data
import re
import ast
from tqdm import tqdm
import os

from post_intentions import post_intention_predict

import pandas as pd

## 加载数据

In [2]:
tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/projects/SpCQL_db_process/test.json", "json")       # 2007
# trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/projects/SpCQL_db_process/train.json", "json")     # 7001

intention_labels = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/intentions/MyMethod_SpCQL_intention_labels_list.json", "json")
intention_predict_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/intentions/MyMethod_SpCQL_intention_predict_list_SpCQL_intention_Chinese-Mistral-7B-Instruct-v0.1-1.5epoch.pickle", "pickle")
intention_predicts = [ intention_predict_list[i][0] for i in range(len(intention_predict_list))]

trians_ft_datas = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/intentions/MyMethod_SpCQL_ft.json", "json")

def get_restrict_name( text ):
    # 匹配单引号内的文字
    matches = re.findall(r"'(.*?)'", text)
    # 输出结果
    return matches

def exact_equal( intention_predict_post, intention_labels, errors):
    obj_not_equ = []
    rest_not_equ = []
    ret_not_equ = []
    total_equal = []
    for i in range(len(intention_predict_post)):
        obj_not_equ.append( str(intention_predict_post[i]["对象"]).lower() == str(intention_labels[i]["对象"]).lower() )
        rest_not_equ.append( set( get_restrict_name( str(intention_predict_post[i]["约束"]).lower() ) ) == set( get_restrict_name( str(intention_labels[i]["约束"]).lower() ) ))
        ret_not_equ.append( str(intention_predict_post[i]["返回形式"]).lower() == str(intention_labels[i]["返回形式"]).lower() )
        total_equal.append( str(intention_predict_post[i]).lower() == str(intention_labels[i]).lower() )
    print( f"总数：{len(obj_not_equ)}，解析错误数量：{len(errors)}，完全一致的数量：{sum(total_equal)}，对象一致的数量：{sum(obj_not_equ)}，约束一致的数量：{sum(rest_not_equ)}，返回格式一致的数量：{sum(ret_not_equ)}")
    return total_equal, obj_not_equ, rest_not_equ, ret_not_equ

In [4]:
intention_predict_post, errors = post_intention_predict( intention_predicts )
total_equal, obj_not_equ, rest_not_equ, ret_not_equ = exact_equal( intention_predict_post, intention_labels, errors)        # 总数：2007，解析错误数量：0，一致的数量：1227，
total_er_index = []
for i in range(len(rest_not_equ)):
    if not rest_not_equ[i]:
        total_er_index.append( i )
# total_er_index

总数：2007，解析错误数量：0，完全一致的数量：816，对象一致的数量：1227，约束一致的数量：1846，返回格式一致的数量：1406


In [5]:
save_data( intention_predict_post, "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/ablation/non_aligned_intention_predicts_list.json")

## 对齐数据库

In [6]:
from select_subgraph import SelectSubgraph, check_key, merge_key
from align_db_by_subgraph import subgraph_align
from collections import OrderedDict
from wayne_utils import load_data, save_data

tests_query_dict = load_data( "db_instance/align_dict_tests.json", "json")
trains_query_dict = load_data( "db_instance/align_dict_trains.json", "json")
all_query_dict = OrderedDict({})
all_query_dict = merge_key( all_query_dict, tests_query_dict)
all_query_dict = merge_key( all_query_dict, trains_query_dict)
# len(all_query_dict)     # 1980 + 6857 = 8837          # 9008 = 2007 +7001 
os.environ["CUDA_VISIBALE_DEVICES"] = "5"

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
"""获取top1相似度的子图，准确率在94%
Top1匹配率：0.9407075236671649
Top2匹配率：0.9561534628799203
Top3匹配率：0.968609865470852
"""
top1_list = SelectSubgraph( intention_predict_post, all_query_dict)                     

Embedding all query dict
An unexpected error occurred.


Matching subgraph: 100%|██████████| 2007/2007 [28:22<00:00,  1.18it/s]


Top1匹配率：0.9407075236671649
Top2匹配率：0.9561534628799203
Top3匹配率：0.968609865470852


In [8]:
aligned_intention_list = subgraph_align( intention_predict_post, top1_list)             # 用这个子图对齐后处理结果

"""意图一致度评估
# 原始预测结果：总数：2007，解析错误数量：0，完全一致的数量：816，对象一致的数量：1227，约束一致的数量：1846，返回格式一致的数量：1406
# Edit-Distance：无包含：总数：2007，解析错误数量：0，完全一致的数量：888，对象一致的数量：1317，约束一致的数量：1899，返回格式一致的数量：1406
# Edit-Distance：>2 包含：总数：2007，解析错误数量：0，完全一致的数量：897，对象一致的数量：1325，约束一致的数量：1903，返回格式一致的数量：1406
# Edit-Distance：>3 包含：总数：2007，解析错误数量：0，完全一致的数量：898，对象一致的数量：1325，约束一致的数量：1903，返回格式一致的数量：1406
# Similarity：总数：2007，解析错误数量：0，完全一致的数量：901，对象一致的数量：1418，约束一致的数量：1938，返回格式一致的数量：1406
# 完美subgraph 总数：2007，解析错误数量：0，完全一致的数量：1174，对象一致的数量：1747，约束一致的数量：1938，返回格式一致的数量：1406
# Top1 (94%) subgraph 总数：2007，解析错误数量：0，完全一致的数量：1120，对象一致的数量：1670，约束一致的数量：1936，返回格式一致的数量：1406
"""
total_equal, obj_not_equ, rest_not_equ, ret_not_equ = exact_equal( aligned_intention_list, intention_labels, errors)    # 评估意图

Align:   0%|          | 0/2007 [00:00<?, ?it/s]/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Align: 100%|██████████| 2007/2007 [03:05<00:00, 10.80it/s]

总数：2007，解析错误数量：0，完全一致的数量：1120，对象一致的数量：1670，约束一致的数量：1936，返回格式一致的数量：1406


In [9]:
save_data( aligned_intention_list, "intentions_aligned.json")

In [12]:
def show_not_equal( index ):
    print( tests[index]['query'])
    print( tests[index]['cypher'])
    print( "=========================")
    print( "Label Intention")
    print( intention_labels[index]["对象"])
    print( "                           ")

    print( "=========================")
    print( "Predict Intention")
    print( intention_predict_post[index]["对象"])
    print( "                           ")
    
    print( "=========================")
    print( "Aligned Intention")
    print( aligned_intention_list[index]["对象"])
    print( "                           ")

not_equ_index = []
for i in range(len(obj_not_equ)):
    if not obj_not_equ[i]:
        not_equ_index.append(i)
not_equ_index

[4,
 5,
 27,
 43,
 47,
 49,
 62,
 63,
 68,
 70,
 73,
 76,
 78,
 82,
 90,
 92,
 106,
 108,
 115,
 117,
 127,
 140,
 142,
 165,
 193,
 195,
 196,
 198,
 199,
 207,
 212,
 216,
 222,
 231,
 241,
 242,
 267,
 290,
 291,
 294,
 295,
 306,
 307,
 315,
 316,
 333,
 355,
 360,
 364,
 366,
 370,
 373,
 377,
 397,
 402,
 407,
 408,
 409,
 413,
 415,
 416,
 424,
 439,
 441,
 443,
 457,
 461,
 464,
 466,
 473,
 476,
 477,
 478,
 481,
 486,
 532,
 544,
 550,
 551,
 552,
 561,
 571,
 583,
 599,
 600,
 603,
 605,
 607,
 613,
 614,
 616,
 618,
 626,
 634,
 637,
 638,
 641,
 646,
 658,
 672,
 674,
 679,
 680,
 681,
 684,
 685,
 697,
 699,
 703,
 712,
 719,
 721,
 733,
 748,
 757,
 766,
 774,
 778,
 786,
 787,
 805,
 810,
 815,
 817,
 819,
 822,
 823,
 839,
 840,
 844,
 847,
 850,
 858,
 862,
 866,
 874,
 880,
 888,
 889,
 899,
 913,
 920,
 921,
 923,
 924,
 926,
 928,
 929,
 932,
 936,
 946,
 954,
 958,
 959,
 963,
 965,
 975,
 976,
 981,
 984,
 986,
 990,
 1000,
 1008,
 1015,
 1016,
 1021,
 1033,
 103

In [13]:
show_not_equal( 1703 )

我们经常用到的1-苯基环丁基甲酸的CAS RN：是什么？
match(:ENTITY{name:'1-苯基环丁基甲酸'})-[:Relationship{name:'CASRN'}]-> (p) return p.name
Label Intention
{'name': '1-苯基环丁基甲酸', 'CASRN': '待查询'}
                           
Predict Intention
{'name': '1-苯基环丁基甲酸', 'CAS RN': '待查询'}
                           
Aligned Intention
{'name': '1-苯基环丁基甲酸', 'CASRN': '待查询'}
                           


## 将对齐后的结果转变为骨干

In [9]:
from intention_to_skeleton import get_skeleton_from_intention

In [10]:
predict_skeleton_list = get_skeleton_from_intention( intention_predict_post )
save_data( predict_skeleton_list, "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/ablation/non_aligned_intention_skeleton_list.json")
'''
train_intentions = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/intentions/MyMethod_SpCQL_intention_trains_list.json", "json")
train_skeleton_list = get_skeleton_from_intention( train_intentions )
save_data( train_skeleton_list, "train_skeleton_list.json")
label_skeleton_list = get_skeleton_from_intention( intention_labels )
save_data( label_skeleton_list, "label_skeleton_list.json")'''

'\ntrain_intentions = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/intentions/MyMethod_SpCQL_intention_trains_list.json", "json")\ntrain_skeleton_list = get_skeleton_from_intention( train_intentions )\nsave_data( train_skeleton_list, "train_skeleton_list.json")\nlabel_skeleton_list = get_skeleton_from_intention( intention_labels )\nsave_data( label_skeleton_list, "label_skeleton_list.json")'